In [1]:
from keras.models import model_from_json, Sequential, Model, clone_model
from keras.layers import Input, Conv2D, InputLayer, MaxPooling2D, BatchNormalization, Activation, Dense
import numpy as np
import pandas as pd
from tools.tools import *
import tensorflow as tf

/home/herval/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load data

X_train_original = pd.read_csv('/media/herval/Save/School/THB/Master/Semestre3/Projekt/data/processed/all_with_augmented/X_train.csv', header=None).values[:, 1:]
Y_train_original = pd.read_csv('/media/herval/Save/School/THB/Master/Semestre3/Projekt/data/processed/all_with_augmented/Y_train.csv', header=None).values[:, 1]
X_test_original = pd.read_csv('/media/herval/Save/School/THB/Master/Semestre3/Projekt/data/processed/all_with_augmented/X_test.csv', header=None).values[:, 1:]
Y_test_original = pd.read_csv('/media/herval/Save/School/THB/Master/Semestre3/Projekt/data/processed/all_with_augmented/Y_test.csv', header=None).values[:, 1]

In [3]:
X_train = X_train_original.reshape((X_train_original.shape[0], 40, 40, 3)) / 255.
X_test = X_test_original.reshape((X_test_original.shape[0], 40, 40, 3)) / 255.

Y_train = Y_train_original
Y_test = Y_test_original

In [2]:
# Load model

json_file = open('./detector/models/classifier/class_model_train_loss-0.00899300720441_train_acc-0.990210576127_test_loss-0.151853362135_test_acc-0.829227417503_162201811834.json', 'r')
model40_json = json_file.read()
json_file.close()
model40 = model_from_json(model40_json)
model40.load_weights("./detector/models/classifier/class_weights_train_loss-0.00899300720441_train_acc-0.990210576127_0.1518533621350.82922741750316220181.h5")
model40.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 40, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 46, 46, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 44, 44, 32)        896       
_________________________________________________________________
bn0 (BatchNormalization)     (None, 44, 44, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 44, 44, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 22, 22, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 20, 20, 64)        18496     
__________

In [5]:
index = 910
example = convertSavedMatrixToApplicableMatrixOfImages(X_test_original, (40, 40), (40, 40))[index]
image_example = Image.fromarray(example, 'RGB')
print("Real: {}".format(Y_test[index]))
print("Predicted: {}".format(model40.predict(np.array([X_test[index]]))))
image_example.show()

Real: 1
Predicted: [[0.27976918]]


In [6]:
#model40_copy = clone_model(model40)

In [3]:
input20 = Input(shape = (22, 22, 3))

X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv20')(input20)
X = BatchNormalization(axis = 3, name = 'bn20')(X)
X = Activation('relu')(X)
X = MaxPooling2D((2, 2), name='max_pool20')(X)
X = model40.get_layer('conv1')(X)

tf.get_variable_scope().reuse_variables()
X = model40.get_layer('bn1')(X)
X = model40.get_layer('activation_2')(X)
X = model40.get_layer('max_pool1')(X)
X = model40.get_layer('conv2')(X)

X = model40.get_layer('bn2')(X)
X = model40.get_layer('activation_3')(X)
X = model40.get_layer('max_pool2')(X)
X = model40.get_layer('flatten_1')(X)
X = Dense(1, activation='sigmoid', name='fc_20')(X)

model20 = Model(inputs = input20, outputs = X)

model20.summary()

model20_json = model20.to_json()
with open("./classificators/models/model20.json", "w") as json_file:
    json_file.write(model20_json)
model20.save_weights("./classificators/weights/model20.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 22, 22, 3)         0         
_________________________________________________________________
conv20 (Conv2D)              (None, 20, 20, 32)        896       
_________________________________________________________________
bn20 (BatchNormalization)    (None, 20, 20, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
max_pool20 (MaxPooling2D)    (None, 10, 10, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               multiple                  18496     
_________________________________________________________________
bn1 (BatchNormalization)     multiple                  256       
__________

In [4]:
input10 = Input(shape = (10, 10, 3))

X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv10')(input10)
X = BatchNormalization(axis = 3)(X)
X = Activation('relu')(X)
X = MaxPooling2D((2, 2), name='max_pool10')(X)
X = model40.get_layer('conv2')(X)

tf.get_variable_scope().reuse_variables()
X = model40.get_layer('bn2')(X)
X = model40.get_layer('activation_3')(X)
X = model40.get_layer('max_pool2')(X)
X = model40.get_layer('flatten_1')(X)
X = Dense(1, activation='sigmoid', name='fc_10')(X)

model10 = Model(inputs = input10, outputs = X)

model10.summary()

model10_json = model10.to_json()
with open("./classificators/models/model10.json", "w") as json_file:
    json_file.write(model10_json)
model20.save_weights("./classificators/weights/model10.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 10, 3)         0         
_________________________________________________________________
conv10 (Conv2D)              (None, 8, 8, 64)          1792      
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
activation_2 (Activation)    (None, 8, 8, 64)          0         
_________________________________________________________________
max_pool10 (MaxPooling2D)    (None, 4, 4, 64)          0         
_________________________________________________________________
conv2 (Conv2D)               multiple                  73856     
_________________________________________________________________
bn2 (BatchNormalization)     multiple                  512       
__________

NameError: name 'model20' is not defined